In [18]:


import pandas as pd
import numpy as np
import os

# Folder path for your CSV exports
folder = "/content/csv_exports"

# Load data
profiles = pd.read_csv(os.path.join(folder, "profiles_psych.csv"))
competencies = pd.read_csv(os.path.join(folder, "competencies_yearly.csv"))
papi = pd.read_csv(os.path.join(folder, "papi_scores.csv"))
performance = pd.read_csv(os.path.join(folder, "performance_yearly.csv"))

profiles.head()


,employee_id,pauli,faxtor,disc,disc_word,mbti,iq,gtq,tiki
0,EMP100000,86,75,SI,Steadiness-Influencer,NaN,94.0,33.0,2
1,EMP100001,48,52,DS,Dominant-Steadiness,INTP,94.0,17.0,3
2,EMP100002,66,38,DC,Dominant-Conscientious,NaN,109.0,20.0,3
3,EMP100003,39,63,SI,Steadiness-Influencer,ENTJ,85.0,39.0,3
4,EMP100004,75,100,NaN,Steadiness-Conscientious,INTJ,134.0,21.0,9


In [19]:
# Clean column names
profiles.columns = profiles.columns.str.strip().str.lower()
competencies.columns = competencies.columns.str.strip().str.lower()
papi.columns = papi.columns.str.strip().str.lower()
performance.columns = performance.columns.str.strip().str.lower()

# Drop duplicates
profiles.drop_duplicates(subset=['employee_id'], inplace=True)
performance.drop_duplicates(subset=['employee_id', 'year'], inplace=True)

# Handle missing values
profiles.fillna(0, inplace=True)
competencies.fillna(0, inplace=True)
papi.fillna(0, inplace=True)
performance.fillna(0, inplace=True)


In [20]:
# Pivot competency scores by pillar
competency_pivot = competencies.pivot_table(index=['employee_id', 'year'],
                                            columns='pillar_code',
                                            values='score',
                                            aggfunc='mean').reset_index()

# Pivot PAPI scales
papi_pivot = papi.pivot_table(index='employee_id',
                              columns='scale_code',
                              values='score',
                              aggfunc='mean').reset_index()


In [21]:
# Merge competency + performance
merged = pd.merge(competency_pivot, performance, on=['employee_id', 'year'], how='inner')

# Merge with profiles and PAPI
merged = pd.merge(merged, profiles, on='employee_id', how='left')
merged = pd.merge(merged, papi_pivot, on='employee_id', how='left')

print(merged.shape)
merged.head()


(10050, 41)


,employee_id,year,CEX,CSI,FTC,GDR,IDS,LIE,QDD,SEA,...,Papi_N,Papi_O,Papi_P,Papi_R,Papi_S,Papi_T,Papi_V,Papi_W,Papi_X,Papi_Z
0,DUP1942,2021,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,...,0.0,3.0,5.0,7.0,9.0,9.0,1.0,8.0,3.0,0.0
1,DUP1942,2022,3.0,3.0,3.0,2.0,2.0,3.0,4.0,3.0,...,0.0,3.0,5.0,7.0,9.0,9.0,1.0,8.0,3.0,0.0
2,DUP1942,2023,2.0,3.0,4.0,3.0,3.0,3.0,4.0,4.0,...,0.0,3.0,5.0,7.0,9.0,9.0,1.0,8.0,3.0,0.0
3,DUP1942,2024,2.0,2.0,2.0,2.0,3.0,2.0,2.0,1.0,...,0.0,3.0,5.0,7.0,9.0,9.0,1.0,8.0,3.0,0.0
4,DUP1942,2025,4.0,5.0,5.0,5.0,5.0,0.0,5.0,5.0,...,0.0,3.0,5.0,7.0,9.0,9.0,1.0,8.0,3.0,0.0


In [22]:
from sklearn.preprocessing import LabelEncoder

# Encode categorical columns
cat_cols = ['disc', 'disc_word', 'mbti']
for col in cat_cols:
    if col in merged.columns:
        merged[col] = LabelEncoder().fit_transform(merged[col].astype(str))

# Drop non-numeric or irrelevant columns
merged = merged.dropna(subset=['rating'])
X = merged.drop(columns=['rating', 'year'])
y = merged['rating']


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# --- FIX HERE ---
# Drop columns that are IDs or non-numeric
exclude_cols = ['employee_id', 'pillar_code', 'year', 'rating']
X = merged.drop(columns=[col for col in exclude_cols if col in merged.columns])

# Keep only numeric columns
X = X.select_dtypes(include=[np.number])
y = merged['rating']

# --- TRAIN/TEST SPLIT ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- MODEL TRAINING ---
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("✅ Model trained successfully!")
print("R² Score:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))


✅ Model trained successfully!
R² Score: 0.04824627206958121
RMSE: 3.2721600747405937


In [27]:
import pandas as pd
import numpy as np

# Create performance class labels
merged['rating_class'] = pd.cut(
    merged['rating'],
    bins=[0, 3, 4.5, 10],
    labels=['Low', 'Average', 'High']
)

# Check balance
print(merged['rating_class'].value_counts())


rating_class
Low        5589
Average    2437
High        894
Name: count, dtype: int64


In [28]:
from sklearn.preprocessing import LabelEncoder

# Encode categorical fields
cat_cols = ['disc', 'disc_word', 'mbti']
for col in cat_cols:
    if col in merged.columns:
        merged[col] = LabelEncoder().fit_transform(merged[col].astype(str))

# Drop non-numeric / identifiers
exclude_cols = ['employee_id', 'pillar_code', 'year', 'rating']
X = merged.drop(columns=[c for c in exclude_cols if c in merged.columns])

# Keep only numeric columns
X = X.select_dtypes(include=[np.number])

# Target variable
y = merged['rating_class']


In [34]:
# Option 1 — safest for HR data: replace NaN with column mean or 0
X = X.fillna(X.mean(numeric_only=True))  # numeric columns only
X = X.fillna(0)
print("Any missing values left?", X.isnull().sum().sum())


Any missing values left? 0


In [31]:
merged = merged.dropna(subset=['rating_class'])
y = merged['rating_class']


In [36]:
print("X shape:", X.shape)
print("y shape:", y.shape)
# Drop rows where rating_class is missing
merged = merged.dropna(subset=['rating_class'])

# Rebuild X and y from the same filtered dataframe
exclude_cols = ['employee_id', 'pillar_code', 'year', 'rating']
X = merged.drop(columns=[c for c in exclude_cols if c in merged.columns])
X = X.select_dtypes(include=[np.number]).fillna(X.mean(numeric_only=True))

y = merged['rating_class']
print("✅ Shapes after fix → X:", X.shape, "| y:", y.shape)


X shape: (10050, 38)
y shape: (8920,)
✅ Shapes after fix → X: (8920, 38) | y: (8920,)


In [37]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model_clf = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    class_weight='balanced'
)
model_clf.fit(X_train, y_train)

y_pred = model_clf.predict(X_test)

print("✅ Classification model trained successfully!")
print("\nClassification Report:\n", classification_report(y_test, y_pred))


✅ Classification model trained successfully!

Classification Report:
               precision    recall  f1-score   support

     Average       0.94      0.95      0.95       487
        High       0.99      0.79      0.88       179
         Low       0.97      0.99      0.98      1118

    accuracy                           0.96      1784
   macro avg       0.97      0.91      0.94      1784
weighted avg       0.96      0.96      0.96      1784



In [38]:
import os
export_folder = "/content/exported_data"
os.makedirs(export_folder, exist_ok=True)

# Feature importance
importances = pd.DataFrame({
    "feature": X.columns,
    "importance": model_clf.feature_importances_
}).sort_values("importance", ascending=False)
importances.to_csv(os.path.join(export_folder, "feature_importance_classification.csv"), index=False)

# Predictions per employee
merged["predicted_class"] = model_clf.predict(X)
merged.to_csv(os.path.join(export_folder, "employee_predictions_classification.csv"), index=False)

print("✅ Exported to /content/exported_data/")


✅ Exported to /content/exported_data/
